# POC

## Imports

In [2]:
!which python3

/home/secouss/repos/fh-industrie/.venv/bin/python3


In [ ]:
import itertools
import json
import os
import re
import tempfile
from pathlib import Path
from typing import Any, Dict, List, Optional

import dotenv
import pandas as pd
import pytesseract
import requests
from openpyxl import load_workbook
from pdf2image import convert_from_path
from PIL import Image
from tqdm import tqdm

## Vars

In [64]:
PATH_DOCS = Path("../docs")
PATH_TEMPLATE = Path("../templates")
PATH_OUTPUTS = Path("../outputs")

## Claude test

In [30]:
dotenv.load_dotenv("../keys.sh", override=True)

python-dotenv could not parse statement starting at line 1


True

In [31]:
CLAUDE_KEY = os.environ["CLAUDE_KEY"]

In [32]:
class ClaudeClient:
    """A simple client for the Anthropic Claude API."""
    
    def __init__(self, api_key: str):
        """Initialize the Claude client with your API key."""
        self.api_key = api_key
        
        self.base_url = "https://api.anthropic.com/v1"
        self.headers = {
            "x-api-key": self.api_key,
            "anthropic-version": "2023-06-01",
            "content-type": "application/json"
        }
    
    def create_message(
        self,
        model: str = "claude-3-7-sonnet-20250219",
        messages: List[Dict[str, str]] = None,
        system: Optional[str] = None,
        max_tokens: int = 1024,
        temperature: float = 0.7,
        stream: bool = False
    ) -> Dict[str, Any]:
        """
        Create a message using the Claude API.
        
        Args:
            model: The Claude model to use
            messages: List of message objects with role and content
            system: Optional system prompt to guide Claude's behavior
            max_tokens: Maximum number of tokens to generate
            temperature: Controls randomness (0-1)
            stream: Whether to stream the response
            
        Returns:
            API response as a dictionary
        """
        if not messages:
            raise ValueError("Messages are required")
            
        url = f"{self.base_url}/messages"
        
        payload = {
            "model": model,
            "messages": messages,
            "max_tokens": max_tokens,
            "temperature": temperature,
            "stream": stream
        }
        
        if system:
            payload["system"] = system
            
        response = requests.post(url, headers=self.headers, json=payload)
        
        if response.status_code != 200:
            raise Exception(f"API request failed with status {response.status_code}: {response.text}")
            
        return response.json()


In [33]:
client = ClaudeClient(CLAUDE_KEY)

In [27]:
system = """
    Renvoie tes réponses sous format json.
    Extrait toutes les informations relatives aux différents parties.
"""

content = "TRIBUNAL JUDICIAIRE DE DIEPPE Service chargé du contrôle des expertises judiciaires Tél. : 02.76.72.97.34 Référence à rappeler dans tous courriers : Mesure d'instruction : 24/173 N° de : 24/00052 N° Portalis DB2U-W-B7I-DDMR Ordonnance de Référé du 05 Juin 2024 DEMANDEURS M. Claude MASSERE Mme Françoise, Brigitte, Jeanne, Julia LAMAILLE épouse MASSERE représentés par la SCP MORIVAL AMISSE MABIRE, Avocats associés au Barreau de DIEPPE DÉFENDEURS La S.A.R.L. BRUGOT XAVIER représentée par la SCP LENGLET, MALBESIN & Associés, Avocats associés au Barreau de ROUEN La S.A. AXA France IARD représentée par la SCP LENGLET, MALBESIN & Associés, Avocats associés au Barreau de ROUEN M. Olivier BOUDET représenté par la SELARL PATRICE LEMIEGRE PHILIPPE FOURDRIN SUNA GUNEY & Associés, Avocats associés au Barreau de ROUEN La Société MUTUELLE DES ARCHITECTES FRANCAIS non représentée Monsieur, DIEPPE, le 21 Août 2024. Monsieur Franck HIBON 6 Impasse des Hautes Terres Les Vertus franck.hibon.expert-de-justice.or2 76550 SAINT AUBIN SUR SCIE EXPERTISE JUDICIAIRE, AVEC CONSIGNATION Date limite de dépôt du rapport d'expertise judiciaire : 28 Février 2025 J'ai l'honneur de vous adresser sous ce pli copie d'une décision qui vous a commis. re IMPORTANT : article 267 du Code de Procédure Civile - Décret n° 89-511 du 20 juillet 1989, article 3, en vigueur le 15 septembre 1989 : Il vous appartient de faire connaître au service chargé du contrôle des expertises judiciaires votre acceptation ou votre refus dès réce fion de la résente. Dans le cas où vous ne pourriez accepter cette mission, veuillez renvoyer l'ensemble des documents au service chargé du contrôle des expertises judiciaires."

messages = [
    {"role": "user", "content": content}
]

response = client.create_message(
    model="claude-3-7-sonnet-20250219",
    messages=messages,
    system=system,
    max_tokens=500,
    temperature=0.7
)

# Print the response
print("Claude's response:")
print(response["content"][0]["text"])

Claude's response:
```json
{
  "type_document": "Ordonnance d'expertise judiciaire",
  "date_document": "21 Août 2024",
  "tribunal": "TRIBUNAL JUDICIAIRE DE DIEPPE",
  "reference": {
    "mesure_instruction": "24/173",
    "numero": "24/00052",
    "portalis": "DB2U-W-B7I-DDMR"
  },
  "decision": "Ordonnance de Référé du 05 Juin 2024",
  "parties": {
    "demandeurs": [
      {
        "nom": "M. Claude MASSERE",
        "representation": "SCP MORIVAL AMISSE MABIRE, Avocats associés au Barreau de DIEPPE"
      },
      {
        "nom": "Mme Françoise, Brigitte, Jeanne, Julia LAMAILLE épouse MASSERE",
        "representation": "SCP MORIVAL AMISSE MABIRE, Avocats associés au Barreau de DIEPPE"
      }
    ],
    "defendeurs": [
      {
        "nom": "S.A.R.L. BRUGOT XAVIER",
        "representation": "SCP LENGLET, MALBESIN & Associés, Avocats associés au Barreau de ROUEN"
      },
      {
        "nom": "S.A. AXA France IARD",
        "representation": "SCP LENGLET, MALBESIN & Associés

## OCR

In [ ]:
def ocr_pdf(pdf_path, pages : Optional[List[int]] = None, language='eng', dpi=300) -> List[str]:
    """
    Performs OCR on a PDF and saves the text to a file.
    
    Args:
        pdf_path (str): Path to the PDF file
        output_path (str, optional): Path to save the output text. If None, uses the PDF name with .txt extension
        language (str, optional): Language for OCR. Default is 'eng'
        dpi (int, optional): DPI for rendering PDF. Higher is better quality but slower.
    """
    
    # Create temp directory for storing images
    with tempfile.TemporaryDirectory() as temp_dir:
        
        # Convert PDF to images
        try:
            images = convert_from_path(pdf_path, dpi=dpi)
            print(f"PDF converted to {len(images)} images.")
        except Exception as e:
            print(f"Error converting PDF: {e}")
            return None
        
        if pages is None:
            pages = list(range(len(images)))
        
        # Process each page
        text_per_page = []
        for i, image in tqdm(list(enumerate(images)), desc="OCR pages"):
            page_number = i + 1
            if not page_number in pages:
                continue
            print(f"OCR processing page {page_number}/{len(images)}...")
            
            # Perform OCR
            text = pytesseract.image_to_string(image, lang=language)
            text_per_page.append(text)
        
    return text_per_page

In [38]:
pdf_path = PATH_DOCS / "0- ARBORESCENCE DOSSIERS JUD/EXPERT/01 - Notes aux parties/Note n┬░1 - Visio Adm/TJ DIEPPE - MASSERE - Ordonnance du 05 06 2024.pdf"
assert pdf_path.exists()

In [39]:
text_per_page = ocr_pdf(pdf_path=pdf_path)
text = text_per_page[0]

PDF converted to 10 images.
OCR processing page 1/10...
OCR processing page 2/10...
OCR processing page 3/10...
OCR processing page 4/10...
OCR processing page 5/10...
OCR processing page 6/10...
OCR processing page 7/10...
OCR processing page 8/10...
OCR processing page 9/10...


In [143]:
text = "\n---------------------------------------\n".join(text_per_page)
print(text)

TRIBUNAL JUDICIAIRE

DE DIEPPE

Service chargé du contr6éle des expertises judiciaires
Tél. : 02.76.72.97.34

Référence 4 rappeler dans tous courriers :

Mesure d’instruction : 24/173

Monsieur Franck HIBON
N° de R.G. : 24/00052 6 Impasse des Hautes Terres
N° Portalis DB2U-W-B7I-DDMR Les Vertus
franck.hibon @expert-de-justice.org
Ordonnance de Référé du 05 Juin 2024 76550 SAINT AUBIN SUR SCIE
DEMANDEURS EXPERTISE JUDICIAIRE, AVEC CONSIGNATION
M. Claude MASSERE
Mme Francoise, Brigitte, Jeanne, Julia Date limite de dépét du rapport d’expertise judiciaire :

LAMAILLE épouse MASSERE

28 Février 2025

représentés par la SCP MORIVAL AMISSE
MABIRE, Avocats associés au Barreau de DIEPPE

DEFENDEURS

La S.A.R.L. BRUGOT XAVIER

représentée par la SCP LENGLET, MALBESIN &
Associés, Avocats associés au Barreau de ROUEN

La S.A. AXA France IARD

représentée par la SCP LENGLET, MALBESIN &
Associés, Avocats associés au Barreau de ROUEN

M. Olivier BOUDET

représenté par la SELARL PATRICE LEMIEGRE
PHIL

## XLSX

In [88]:
path_input = PATH_TEMPLATE / "feuille_présence.xlsx"
path_output = PATH_OUTPUTS / "feuille_presence.xlsx"
assert path_input.exists() and path_output.parent.exists()

### Pandas

In [89]:
df = pd.read_excel(path_input, header=None)
df

,0,1,2,3,4,5,6
0,{instruction:colonne},FEUILLE DE PRESENCE,NaN,NaN,NaN,NaN,NaN
1,NaN,TJ DE DIEPPE - Ordonnance de référé du {date_o...,NaN,NaN,NaN,NaN,NaN
2,NaN,{demendeurs} c/ {defenderesses} & Autres ...,NaN,NaN,NaN,NaN,NaN
3,NaN,LIEU DE L'EXPERTISE : {lieu_expertise},NaN,NaN,NaN,NaN,NaN
4,NaN,DATE DE LA RÉUNION : {date_reunion},NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Partie,Nom,Réf. dossier,Téléphone,E.Mail (utilisé dans OPALEXE pour les Conseils),Visa
8,{instruction:list:demandeur},Demandeurs,{demendeur:nom},NaN,NaN,{demandeur:email},NaN
9,NaN,Conseil {demandeur:nom_abrege} \n,{demandeur:profession_conseil} {demandeur:nom...,NaN,{demandeur:téléphones_conseil},{demandeur:email_conseil},NaN


In [90]:
df.to_excel(path_output, index=False, header=False)

### Openpyxl

In [138]:
class ExcelManager:

    def __init__(self, path_excel):
        self.wb = load_workbook(path_excel)
        self.ws = self.wb.worksheets[0]

    def replace_content(self, infos : dict, verbose : bool = True) -> None:
        n = 100

        changes = 0

        for row, column in itertools.product(range(1, n), range(1, n)):
            # read
            content = self.ws.cell(row, column).value

            if content is None:
                continue

            # replace
            for key, value in infos.items():
                new_content = content.replace("{" + key + "}", value)
                if content != new_content:
                    changes += 1
                content = new_content

            # write
            self.ws.cell(row, column, value=content)
    
        if verbose:
            print(f"Excel changes : {changes}")

    def save(self, path_excel) -> None:
        self.wb.save(path_excel)

In [139]:
infos = {
    "date_ordonnance" : "date bar",
    "numero_rg" : "rg foo",
    "lieu_expertise" : "lieu toto",
    "date_reunion" : "date baz"
}

In [140]:
excel = ExcelManager(path_input)
excel.replace_content(infos)
excel.save(path_output)

Excel changes : 4


## Test

In [ ]:
system = """
    Extrait toutes les informations relatives aux différents partis.
    Renvoie tes réponses sous format json :
    {
        "date_ordonnance" : "...",
        "numero_rg" : "...",
        "lieu_expertise" : "...",
        "date_reunion" : "..."
    }
"""

messages = [
    {"role": "user", "content": text}
]

response = client.create_message(
    model="claude-3-7-sonnet-20250219",
    messages=messages,
    system=system,
    max_tokens=1000,
    temperature=0.7
)

# Print the response
print("Claude's response:")
print(response["content"][0]["text"])

Claude's response:


Je vais extraire les informations demandées à partir du document.

```json
{
    "date_ordonnance": "05 Juin 2024",
    "numero_rg": "24/00052",
    "lieu_expertise": "15 Rue des Bords de Mer BELLEVILLE SUR MER - 76370 PETIT-CAUX",
    "date_reunion": "Non précisée dans le document"
}
```


In [191]:
text_infos = response["content"][0]["text"]
res = re.search(pattern="```json(.*)```", string=text_infos, flags=re.DOTALL)
infos = json.loads(res.group(1))
infos

{'date_ordonnance': '05 Juin 2024',
 'numero_rg': '24/00052',
 'lieu_expertise': '15 Rue des Bords de Mer BELLEVILLE SUR MER - 76370 PETIT-CAUX',
 'date_reunion': 'Non précisée dans le document'}

In [193]:
excel = ExcelManager(path_input)
excel.replace_content(infos)
excel.save(path_output)

Excel changes : 4
